In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv
/kaggle/input/santander-customer-transaction-prediction/train.csv
/kaggle/input/santander-customer-transaction-prediction/test.csv


In [2]:
sct=pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
sct_new=pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [3]:
sct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [4]:
print(sct.columns)

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)


In [5]:
sct['target'].value_counts()

target
0    179902
1     20098
Name: count, dtype: int64

In [6]:
sct['ID_code'].value_counts()

ID_code
train_199960    1
train_199961    1
train_199962    1
train_199963    1
train_199964    1
               ..
train_3         1
train_4         1
train_5         1
train_6         1
train_7         1
Name: count, Length: 200000, dtype: int64

In [7]:
sct['var_0'].value_counts()

var_0
13.0656    11
8.6649     11
10.6829    11
9.5114     10
10.9468    10
           ..
9.1800      1
7.3502      1
11.5106     1
14.5987     1
14.1869     1
Name: count, Length: 94672, dtype: int64

In [8]:
sct['var_1'].value_counts()

var_1
-2.4313    10
-2.1515    10
-1.1853    10
-2.5753    10
-7.2808     9
           ..
 0.6422     1
-9.9800     1
 2.6562     1
-0.7863     1
-7.9881     1
Name: count, Length: 108932, dtype: int64

In [9]:
sct_new.columns

Index(['ID_code', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5',
       'var_6', 'var_7', 'var_8',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=201)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
y=sct[['target']]
X=sct.drop(['target','ID_code'],axis=1)
X_new=sct_new.drop(['ID_code'],axis=1)
X_transformed=StandardScaler().fit_transform(X)
X_new_transformed=StandardScaler().fit_transform(X_new)
X_temp,X_val,y_temp,y_val=train_test_split(X_transformed,y,test_size=0.05,random_state=1)
X_train,X_test,y_train,y_test=train_test_split(X_temp,y_temp,test_size=0.05,random_state=2)

In [11]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input,Dense,Normalization
from keras.optimizers import SGD,AdamW
from keras.callbacks import EarlyStopping
import keras_tuner as kt

2024-07-14 07:15:23.597730: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 07:15:23.597896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 07:15:23.734615: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=10, max_value=100, default=50)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    weight_decay=hp.Float("weight_decay", min_value=0.001, max_value=0.01,sampling="log")
    optimizer = hp.Choice("optimizer", values=["nag", "adamw"])
    if optimizer == "nag":
        optimizer = SGD(learning_rate=learning_rate,momentum=0.9,nesterov=True)
    else:
        optimizer = AdamW(learning_rate=learning_rate,weight_decay=weight_decay)        
    model=keras.Sequential()
    model.add(Input(shape=X_train.shape[1:]))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation="swish",kernel_initializer='he_normal'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=['accuracy'])
    return model

In [13]:
random_search=kt.RandomSearch(build_model,objective='val_accuracy',seed=3,max_trials=5)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
try: random_search.search(X_train,y_train,epochs=10,validation_data=(X_val,y_val),callbacks=[early_stopping])
except kt.errors.FatalError: print("The search is terminated.")

Trial 5 Complete [00h 08m 03s]
val_accuracy: 0.906000018119812

Best val_accuracy So Far: 0.906000018119812
Total elapsed time: 00h 17m 53s


In [14]:
best_model=random_search.get_best_models(3)[0]
print(best_model)

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 66 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 64 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<Sequential name=sequential, built=True>


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 138 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
best_hyper=random_search.get_best_hyperparameters(3)[0]
print(best_hyper.values)

{'n_hidden': 31, 'n_neurons': 72, 'learning_rate': 0.0017611785596585996, 'weight_decay': 0.0062135540967083866, 'optimizer': 'nag'}


In [16]:
best_trial = random_search.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 0 summary
Hyperparameters:
n_hidden: 31
n_neurons: 72
learning_rate: 0.0017611785596585996
weight_decay: 0.0062135540967083866
optimizer: nag
Score: 0.906000018119812


In [17]:
best_model.fit(X_train,y_train,callbacks=[early_stopping],validation_data=(X_val,y_val))

5641/5641 ━━━━━━━━━━━━━━━━━━━━ 30s 4ms/step - accuracy: 0.8983 - loss: 0.3289 - val_accuracy: 0.9060 - val_loss: 0.3119


In [18]:
best_model.evaluate(X_test,y_test)

297/297 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8980 - loss: 0.3296


[0.3243914246559143, 0.9003157615661621]

In [19]:
y_pred=best_model.predict(X_new_transformed)

6250/6250 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step


In [20]:
print(y_pred[:10])

[[0.09928376]
 [0.09929374]
 [0.09929295]
 [0.0992899 ]
 [0.09928681]
 [0.09928782]
 [0.09929267]
 [0.09927862]
 [0.09928615]
 [0.09928743]]


In [21]:
output=pd.DataFrame({'ID_code':sct_new.ID_code,'target':y_pred.ravel()})
output.to_csv('sct_submission.csv',index=False)